In [1]:
import pandas as pd
import numpy as np
import pathlib
from time import time
import warnings
import shutil
from xlrd import XLRDError
warnings.filterwarnings("ignore")

In [2]:
ptr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG', 'RESULT','TEST_TXT',
                  'LO_LIMIT', 'HI_LIMIT', 'UNITS']
ftr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG','TEST_TXT']
prr_df_columns = ['PART_ID', 'PART_FLG', 'NUM_TEST']
mir_df_columns = ['LOT_ID','PART_TYP','JOB_NAM','OPER_NAM','TSTR_TYP','EXEC_VER','TST_TEMP','PKG_TYP','RTST_COD']
config_df_columns = ['TEST_NUM','TEST_TXT',"T_Type","LO_LIMIT",'HI_LIMIT',"MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT",'UNITS']
final_df_columns = ['TEST_NUM', 'TEST_TXT','PART_ID', 'Status',"LO_LIMIT",'HI_LIMIT',"MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT", 'UNITS', 'P_Type']
non_exe_columns = ["TEST_NUM",'TEST_TXT', "SITE_NUM","TEST_FLG","RESULT","LO_LIMIT","HI_LIMIT","MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT","UNITS","P_Type","PART_ID","STATUS"]

In [3]:
def get_FTR_df(masterfile):
    flag = 0
    try:
        df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    except XLRDError:
        print ("FTR Sheet not exists")
        df = pd.DataFrame(columns = ftr_df_columns)
        flag = 1
    return df,flag

In [4]:
def getDataset(masterfile,config_file):
    # Reading all files
    ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]

    #ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    ftr_df,flag = get_FTR_df(masterfile)
    prr_df = pd.read_excel(masterfile, sheet_name = 'PRR')[prr_df_columns]
    mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]    
    config_df = pd.read_csv(config_file)
    #print('config_df :',config_df)
    # empty data frame template
    df_final = pd.DataFrame(columns=final_df_columns)
    non_exe_df = pd.DataFrame(columns=non_exe_columns)
    return ptr_df, ftr_df, prr_df,mir_df,config_df, df_final, non_exe_df,flag

In [5]:
def getExpandReport(df_final, prr_df, config_df):
    # Expand result and will bring PART_ID in final data 
    for row in prr_df.itertuples():
        # PART_FLG == 0 denotes those record is pass in PRR data
        if row.PART_FLG == 0:         
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID
            part_1['Status'] = 'Pass'
            part_1['TEST_TXT'] = config_df['TEST_TXT']
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            # part_1['MAR_LO_LIMIT'] = config_df['MAR_LO_LIMIT']
            # part_1['MAR_HI_LIMIT'] = config_df['MAR_HI_LIMIT']
            part_1['MAR_HI_UP_LIMIT'] = config_df['MAR_HI_UP_LIMIT']
            part_1['MAR_HI_DOWN_LIMIT'] = config_df['MAR_HI_DOWN_LIMIT']
            part_1['MAR_LO_UP_LIMIT'] = config_df['MAR_LO_UP_LIMIT']
            part_1['MAR_LO_DOWN_LIMIT'] = config_df['MAR_LO_DOWN_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            df_final = df_final.append(part_1)
        else:        
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID
            part_1['Status'] = 'Fail'
            part_1['TEST_TXT'] = config_df['TEST_TXT']
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            # part_1['MAR_LO_LIMIT'] = config_df['MAR_LO_LIMIT']
            # part_1['MAR_HI_LIMIT'] = config_df['MAR_HI_LIMIT']
            part_1['MAR_HI_UP_LIMIT'] = config_df['MAR_HI_UP_LIMIT']
            part_1['MAR_HI_DOWN_LIMIT'] = config_df['MAR_HI_DOWN_LIMIT']
            part_1['MAR_LO_UP_LIMIT'] = config_df['MAR_LO_UP_LIMIT']
            part_1['MAR_LO_DOWN_LIMIT'] = config_df['MAR_LO_DOWN_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            #print("before : \n",part_1.iloc[0:row.NUM_TEST,[2]])
            #print("Part_ID : ",row.PART_ID)
            part_1.iloc[0:row.NUM_TEST,[2]] = 'Pass'      # causing issue
            #print("part_1.iloc[0:row.NUM_TEST,[2]] : \n",part_1.iloc[0:row.NUM_TEST,[2]])
            
            df_final = df_final.append(part_1)
    return df_final

In [6]:
def updatePartId(prr_df, ftr_df, ptr_df):
    ftr_counter = 0
    ptr_counter = 0
    ftr_df['PART_ID'] = 0
    ptr_df['PART_ID'] = 0
    # Updating PATR_ID in both FTR and PTR datas
    for row in prr_df.itertuples():
        print("row: ",row)
        if not pd.isnull(row.FTR):
            ftr_df.iloc[ftr_counter:ftr_counter+int(row.FTR),-1] = row.PART_ID
            ftr_counter += int(row.FTR)
        if not pd.isnull(row.PTR):
            ptr_df.iloc[ptr_counter:ptr_counter+int(row.PTR),-1] = row.PART_ID
            print("Result : ",ptr_df.iloc[ptr_counter:ptr_counter+int(row.PTR),-1])
            ptr_counter += int(row.PTR)
    return ftr_df, ptr_df

In [7]:
def fillMissedData(df,config_df):
    # Filling LO_LIMIT, HI_LIMIT, UNITS NaN with existing Configure File
    df.LO_LIMIT = df.LO_LIMIT.mask(df.LO_LIMIT.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').LO_LIMIT))

    df.HI_LIMIT = df.HI_LIMIT.mask(df.HI_LIMIT.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').HI_LIMIT))

    # df.MAR_LO_LIMIT = df.MAR_LO_LIMIT.mask(df.MAR_LO_LIMIT.isna()).fillna(    
    # df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_LO_LIMIT))

    # df.MAR_HI_LIMIT = df.MAR_HI_LIMIT.mask(df.MAR_HI_LIMIT.isna()).fillna(    
    # df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_HI_LIMIT))
    df.MAR_HI_UP_LIMIT = df.MAR_HI_UP_LIMIT.mask(df.MAR_HI_UP_LIMIT.isna()).fillna(
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_HI_UP_LIMIT))
    df.MAR_HI_DOWN_LIMIT = df.MAR_HI_DOWN_LIMIT.mask(df.MAR_HI_DOWN_LIMIT.isna()).fillna(
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_HI_DOWN_LIMIT))
    df.MAR_LO_UP_LIMIT = df.MAR_LO_UP_LIMIT.mask(df.MAR_LO_UP_LIMIT.isna()).fillna(
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_LO_UP_LIMIT))
    df.MAR_LO_DOWN_LIMIT = df.MAR_LO_DOWN_LIMIT.mask(df.MAR_LO_DOWN_LIMIT.isna()).fillna(
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').MAR_LO_DOWN_LIMIT))

    df.UNITS = df.UNITS.mask(df.UNITS.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').UNITS))
      
    return df

In [8]:
def updateResult(df):
    # Updating Fail Pass result for FTR 
    df.loc[(df['P_Type']=="FTR")&(df['RESULT']==1),"STATUS"] = 'Pass'
    df.loc[(df['P_Type']=="FTR")&(df['RESULT']!=1),"STATUS"] = 'Fail'
    # Updating Fail Pass result for PTR 
    #MAR_HI_UP_LIMIT
    df.loc[(df['P_Type']=="PTR")&((df['RESULT']<=df.MAR_HI_DOWN_LIMIT)&(df['RESULT']>=df.MAR_LO_UP_LIMIT)),"STATUS"] = 'Pass'
    df.loc[(df['P_Type']=="PTR")&((df['RESULT']>=df.MAR_HI_UP_LIMIT)|(df['RESULT']<=df.MAR_LO_DOWN_LIMIT)),"STATUS"]='Fail'
#     df.loc[(df['P_Type']=="PTR")&((df['RESULT']>=df.LO_LIMIT)&
#            (df['RESULT']<=df.HI_LIMIT)),"STATUS"] = 'Pass'
#     df.loc[(df['P_Type']=="PTR")&((df['RESULT']<df.LO_LIMIT)&
#     (df['RESULT']>df.HI_LIMIT)),"STATUS"] = 'Fail'

    df.loc[(df['P_Type']=="PTR")&(((df['RESULT']>df.HI_LIMIT)&(df['RESULT']<df.MAR_HI_UP_LIMIT))|
    ((df['RESULT']<df.LO_LIMIT)&(df['RESULT']>df.MAR_LO_DOWN_LIMIT))) ,"STATUS"] = 'Marginal_Fail'

    df.loc[(df['P_Type']=="PTR")&(((df['RESULT']<=df.HI_LIMIT)&(df['RESULT']>df.MAR_HI_DOWN_LIMIT))|
    ((df['RESULT']>=df.LO_LIMIT)&(df['RESULT']<df.MAR_LO_UP_LIMIT))) ,"STATUS"] = 'Marginal_Pass'

    return df

In [9]:
def getNonExeDf(df,non_exe_df):
    df = df.query('Status == "Fail"')
    non_exe_df['TEST_NUM'] = df['TEST_NUM']
    non_exe_df['TEST_TXT'] = df['TEST_TXT']
    non_exe_df['PART_ID'] = df['PART_ID']
    non_exe_df['STATUS'] = 'Non_Executed'
    non_exe_df['P_Type'] = df['Record_type']
    return non_exe_df

In [10]:
def prepareData(masterfile,config_file,IntermediatePath,OutputPath):
    # get All datas and data templates
    ptr_df, ftr_df, prr_df,mir_df, config_df, df_final, non_exe_df,flag = getDataset(masterfile,config_file)
    print('LotId',mir_df.LOT_ID[0])
    print('partType',mir_df.PART_TYP[0])
    LotFolderName = str(mir_df.LOT_ID[0])+'_'+str(mir_df.PART_TYP[0])
    DeviceName = str(mir_df.PART_TYP[0])
    # initilize colums
    ftr_df[['RESULT', 'LO_LIMIT', 'HI_LIMIT',"MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT", 'UNITS']] = np.NaN
    
    # Assuming all result values is 0 initially it denotes FAIL
    ftr_df['RESULT'] = 0
    # Adding types of test in P_Type column 
    ptr_df['P_Type'] = 'PTR'
    ftr_df['P_Type'] = 'FTR'
    # Creating Result column and mark it as 1 where test_flg is 0
    ftr_df.loc[ftr_df['TEST_FLG'] == 0, 'RESULT'] = 1
    # Expanding Report for all part ids
    df_final = getExpandReport(df_final, prr_df, config_df)
    df_final.to_csv(IntermediatePath+"df_final.csv",index=False)
    # print(f'df_final {df_final.shape}')
    # update Record_type PTR or FTR
    op_df = df_final
    op_df["Record_type"] = op_df["TEST_NUM"].map(config_df[['TEST_NUM','T_Type']].set_index('TEST_NUM')['T_Type'],)
    op_df.to_excel(IntermediatePath+"Record_types.xlsx",index=False)
    print(f'op_df {op_df.shape}')
    # updating FTR and PTR counts in PRR data
    if flag == 1:
        prr_df[["PTR"]] = op_df.loc[op_df['Status']=="Pass"].groupby('PART_ID')['Record_type'].value_counts().unstack()[["PTR"]].values
        prr_df['FTR']=np.NaN
    else:
        prr_df[["FTR","PTR"]] = op_df.loc[op_df['Status']=="Pass"].groupby('PART_ID')['Record_type'].value_counts().unstack()[["FTR","PTR"]].values
    prr_df.to_excel(IntermediatePath+"Mapped_PRR.xlsx",index=False)
    ftr_df, ptr_df = updatePartId(prr_df, ftr_df, ptr_df)
    # Merging FTR and PTR together & sorting based on TEST_NUM
    complete_df = ftr_df.append(ptr_df)
    output_df = complete_df.groupby('PART_ID').apply(lambda x: x.sort_values('TEST_NUM'))
    output_df.to_excel(IntermediatePath+"Final_output.xlsx",index=False)
    print(f'output_df {output_df.shape}')
    df = fillMissedData(output_df,config_df)
    df = updateResult(df)
    #df = updateResult(output_df)
    print(f'df {df.shape}')
    # creating Non executed Records
    non_exe_df = getNonExeDf(op_df,non_exe_df)
    non_exe_df = fillMissedData(non_exe_df,config_df)
    df = df.append(non_exe_df)
    print(f'non_exe_df {non_exe_df.shape}')
    df.to_csv(IntermediatePath+"CheckingforFTR.csv",index=False)
    hf = pd.DataFrame()
    df['Combine'] = df['TEST_NUM'].astype(str) + df['STATUS']
    df[['PART_ID','TEST_NUM','Combine']].groupby(['PART_ID','TEST_NUM']).sum().unstack().to_csv(IntermediatePath+"TransposedData.csv")
    #shutil.copy(IntermediatePath+"TransposedData.csv",OutputPath+"TransposedData.csv")
    #df.pivot(index='PART_ID',columns='TEST_NUM',values=['STATUS'])
    return LotFolderName,DeviceName

In [11]:
def POC_Start(ConfigPath,IntermediatePath,OutputPath,infile):
    masterfile = IntermediatePath+infile
    config_file = ConfigPath+'TEST_NUM.csv'
    print("Data Preparation Process Started............")
    LotFolderName = prepareData(masterfile,config_file,IntermediatePath,OutputPath)
    print("Data Preparation Done successfully............")
    return LotFolderName

In [12]:
ConfigPath = r"C:\sunil\Tasks\sai\stdf versions\7March\STDF\STDF\Config/"
IntermediatePath = r"C:\sunil\Tasks\sai\stdf versions\7March\STDF\STDF\Data\Intermediate/"
OutputPath = r"C:\sunil\Tasks\sai\stdf versions\7March\STDF\STDF\Data\Output/"
infile = "lot3.xlsx"
POC_Start(ConfigPath,IntermediatePath,OutputPath,infile)

Data Preparation Process Started............
FTR Sheet not exists
LotId GAL-LOT
partType GOLD8BAR
op_df (119806, 13)
row:  Pandas(Index=0, PART_ID=1, PART_FLG=8, NUM_TEST=1, PTR=1, FTR=nan)
Result :  0    1
Name: PART_ID, dtype: int64
row:  Pandas(Index=1, PART_ID=2, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  1     2
2     2
3     2
4     2
5     2
     ..
70    2
71    2
72    2
73    2
74    2
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=2, PART_ID=3, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  75     3
76     3
77     3
78     3
79     3
      ..
144    3
145    3
146    3
147    3
148    3
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=3, PART_ID=4, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  149    4
150    4
151    4
152    4
153    4
      ..
218    4
219    4
220    4
221    4
222    4
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=4, PART_ID=5, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  223    5
224    5
2

Result :  6623    98
6624    98
6625    98
6626    98
6627    98
        ..
6692    98
6693    98
6694    98
6695    98
6696    98
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=98, PART_ID=99, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  6697    99
6698    99
6699    99
6700    99
6701    99
        ..
6766    99
6767    99
6768    99
6769    99
6770    99
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=99, PART_ID=100, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  6771    100
6772    100
6773    100
6774    100
6775    100
       ... 
6840    100
6841    100
6842    100
6843    100
6844    100
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=100, PART_ID=101, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  6845    101
6846    101
6847    101
6848    101
6849    101
       ... 
6914    101
6915    101
6916    101
6917    101
6918    101
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=101, PART_ID=102, PART_FLG=0, NUM_TEST=72

Result :  12581    187
12582    187
12583    187
12584    187
12585    187
12586    187
12587    187
12588    187
12589    187
12590    187
12591    187
12592    187
12593    187
12594    187
12595    187
12596    187
12597    187
12598    187
12599    187
12600    187
12601    187
12602    187
12603    187
12604    187
12605    187
12606    187
12607    187
12608    187
12609    187
12610    187
12611    187
12612    187
Name: PART_ID, dtype: int64
row:  Pandas(Index=187, PART_ID=188, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  12613    188
12614    188
12615    188
12616    188
12617    188
        ... 
12682    188
12683    188
12684    188
12685    188
12686    188
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=188, PART_ID=189, PART_FLG=8, NUM_TEST=32, PTR=32, FTR=nan)
Result :  12687    189
12688    189
12689    189
12690    189
12691    189
12692    189
12693    189
12694    189
12695    189
12696    189
12697    189
12698    189
12699    189
12700    189
127

Result :  18856    277
18857    277
18858    277
18859    277
18860    277
18861    277
18862    277
18863    277
18864    277
18865    277
18866    277
18867    277
18868    277
18869    277
18870    277
18871    277
18872    277
18873    277
18874    277
18875    277
18876    277
18877    277
18878    277
18879    277
18880    277
18881    277
18882    277
18883    277
18884    277
Name: PART_ID, dtype: int64
row:  Pandas(Index=277, PART_ID=278, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  18885    278
18886    278
18887    278
18888    278
18889    278
        ... 
18954    278
18955    278
18956    278
18957    278
18958    278
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=278, PART_ID=279, PART_FLG=8, NUM_TEST=29, PTR=29, FTR=nan)
Result :  18959    279
18960    279
18961    279
18962    279
18963    279
18964    279
18965    279
18966    279
18967    279
18968    279
18969    279
18970    279
18971    279
18972    279
18973    279
18974    279
18975    279
189

Result :  25358    372
25359    372
25360    372
25361    372
25362    372
        ... 
25427    372
25428    372
25429    372
25430    372
25431    372
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=372, PART_ID=373, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  25432    373
25433    373
25434    373
25435    373
25436    373
        ... 
25501    373
25502    373
25503    373
25504    373
25505    373
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=373, PART_ID=374, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  25506    374
25507    374
25508    374
25509    374
25510    374
        ... 
25575    374
25576    374
25577    374
25578    374
25579    374
Name: PART_ID, Length: 74, dtype: int64
row:  Pandas(Index=374, PART_ID=375, PART_FLG=8, NUM_TEST=62, PTR=62, FTR=nan)
Result :  25580    375
25581    375
25582    375
25583    375
25584    375
        ... 
25637    375
25638    375
25639    375
25640    375
25641    375
Name: PART_ID, Length: 62, dtyp

...

Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1254, PART_ID=1255, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1255, PART_ID=1256, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1256, PART_ID=1257, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1257, PART_ID=1258, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1258, PART_ID=1259, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1259, PART_ID=1260, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1260, PART_ID=1261, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1261, PART_ID=

Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1386, PART_ID=1387, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1387, PART_ID=1388, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1388, PART_ID=1389, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1389, PART_ID=1390, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1390, PART_ID=1391, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1391, PART_ID=1392, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1392, PART_ID=1393, PART_FLG=0, NUM_TEST=72, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1393, PART_ID=

Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1515, PART_ID=1516, PART_FLG=0, NUM_TEST=54, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1516, PART_ID=1517, PART_FLG=0, NUM_TEST=54, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1517, PART_ID=1518, PART_FLG=8, NUM_TEST=1, PTR=1, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1518, PART_ID=1519, PART_FLG=8, NUM_TEST=35, PTR=35, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1519, PART_ID=1520, PART_FLG=0, NUM_TEST=74, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1520, PART_ID=1521, PART_FLG=0, NUM_TEST=54, PTR=74, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1521, PART_ID=1522, PART_FLG=8, NUM_TEST=11, PTR=11, FTR=nan)
Result :  Series([], Name: PART_ID, dtype: int64)
row:  Pandas(Index=1522, PART_ID=15

output_df (54123, 14)
df (54123, 15)
non_exe_df (8444, 15)
Data Preparation Done successfully............


('GAL-LOT_GOLD8BAR', 'GOLD8BAR')